In [ ]:
#import packages
from sklearn.utils import shuffle
import pandas as pd
import csv
from haversine import haversine
import datetime

In [ ]:
#creating dataframe from input CSV
df = pd.DataFrame.from_csv('D:/Nanohub/results/Final results/data_extract/in_STDB_2011_1.csv', sep =',', encoding = 'UTF-8', header = None)
df.columns = ['ip','child_id','datetime','latitude','longitude','parent_id']
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(by = 'child_id', axis = 0, ascending = True, na_position ='first')
#df.drop_duplicates(subset = ['ip','resource_id'], keep = 'first')
doy = []
for date in df['datetime']:
    doy.append(date.dayofyear)
df['DoY'] = doy
df_10 = df[df.groupby('child_id').cumcount(ascending=False) >= 10]


In [ ]:
distinct_resources = list(df_10['child_id'].unique())
#distinct_resources #uncomment to test

In [ ]:
#Implementation of ST_DBSCAN

def ST_DBSCAN(points,max_distance,MinPts,dayRadius):
    global visited 
    visited = []
    noise = []
    c_neighbors = []
    cluster_id = 0
    clusters = []
    in_cluster = []
    for p in points: 
        if p not in visited:
            #print(p)
            # neighbor_points = []
            visited.append(p)
            NeighborPts = regionQuery(p,points,max_distance,dayRadius)
            #print(len(NeighborPts))
            if len(NeighborPts) < MinPts:
                noise.append(p)
            else:
                cluster_id = cluster_id + 1
                g,n = expandCluster(p,NeighborPts,max_distance,MinPts,in_cluster,cluster_id)
                #print(g)
                clusters.append(g)
                #c_neighbors=NeighborPts
                #print(c_neighbors)
                c_neighbors.append(n)
    return clusters, c_neighbors

def regionQuery(p,points,max_distance,dayRadius):
    neighbor_points = []
    loc1 = (p[0],p[1])
    for j in points:
        if j != p:
            # print 'P is %s and j is %s' % (p[0],j[0])
            if abs(p[2]-j[2]) < dayRadius:
                loc2 = (j[0],j[1])
                dist = haversine(loc1, loc2)
                if dist <= max_distance:
                    neighbor_points.append(j)
    neighbor_points.append(p) 
    return neighbor_points   

def expandCluster(p,NeighborPts,max_distance,MinPts,in_cluster,cluster_id):
    in_cluster.append(p)
    cluster = []
    neighbour = []
    t_p = list(p) + [cluster_id]
    cluster.append(t_p)
    for point in NeighborPts:
        if point not in visited:
            visited.append(point)
            new_neighbors = regionQuery(point,points,max_distance,dayRadius)
            if len(new_neighbors) >= MinPts: 
                n_point = list(point) + [cluster_id]
                neighbour.append(n_point)
            if point not in in_cluster:
                 t_point = list(point) + [cluster_id]
                 in_cluster.append(point)
                 cluster.append(t_point)
    return cluster, neighbour


In [ ]:
df_filtered = df_10[df_10['child_id'] == 1]
df_filtered =df_filtered[df_filtered['latitude'] > 0]
points = df_filtered.set_index(['latitude'])[['longitude','DoY','child_id','ip','parent_id']].to_records().tolist()
#points #uncomment to test

In [ ]:
#write to clustered points to two files - cores and neighs

# hyperparameters
max_distance = 50
MaxPts = 10
dayRadius = 7
# end hyperparameters

core, neigh = ST_DBSCAN(points,max_distance,MaxPts,dayRadius)
for l in distinct_resources: #
    df_filtered = df_10[df_10['child_id'] == l]
    #print(df_filtered)
    df_filtered =df_filtered[df_filtered['latitude'] > 0]
    #print(df_filtered.set_index(['latitude'])[['longitude','DoY','resource_id','ip','child_id']].to_records().tolist())
    points = df_filtered.set_index(['latitude'])[['longitude','DoY','child_id','ip','parent_id']].to_records().tolist()
    
    core, neigh = ST_DBSCAN(points,max_distance,MaxPts,dayRadius)
    
    fc_name = 'D:/Nanohub/results/Final Results/cores/cores_2011.csv'
    
    fc = open(fc_name, 'a')
    try:
        writer = csv.writer(fc, lineterminator = '\n')

        for j in range(len(core)):
            for i in range(len(core[j])):
                    writer.writerow( core[j][i] )
    finally:
        fc.close()
    
    fn_name = 'D:/Nanohub/results/Final Results/neighbors/neigh_2011.csv'
    
    fn = open(fn_name, 'a')
    try:
        writer = csv.writer(fn, lineterminator = '\n')

        for j in range(len(neigh)):
            for i in range(len(neigh[j])):
                    writer.writerow( neigh[j][i] )
    finally:
        fn.close()
    